### Getting countries

In [4]:
import requests
import pandas as pd
import json


contest_url = "https://eurovisionapi.runasp.net/api/countries"

contests_response = requests.get(contest_url)

if contests_response.status_code == 200:
    contest = contests_response.json()
    print("Countries retrieved successfully:")

    with open("countries.json", "w", encoding="utf-8") as file:
        json.dump(contest, file, ensure_ascii=False, indent=4)
    
    print("Countries data saved to 'countries.json'")
else:
    print(f"Failed to retrieve data. Status code: {contests_response.status_code}")


Countries retrieved successfully:
Countries data saved to 'countries.json'


### Getting contests

In [5]:
import requests
import pandas as pd
import json


contest_url = "https://eurovisionapi.runasp.net/api/contests"

contests_response = requests.get(contest_url)

if contests_response.status_code == 200:
    contest = contests_response.json()
    print("Contests retrieved successfully:")

    with open("contests.json", "w", encoding="utf-8") as file:
        json.dump(contest, file, ensure_ascii=False, indent=4)
    
    print("Contest data saved to 'contests.json'")
else:
    print(f"Failed to retrieve data. Status code: {contests_response.status_code}")


Contests retrieved successfully:
Contest data saved to 'contests.json'
